In [1]:
%load_ext autoreload

In [2]:
# Utilities 
from options.options import is_notebook, get_options, update_options, print_options
import argparse
import math
import time
import random
import os
from tqdm import tqdm

# Data
import numpy as np
from sklearn.model_selection import train_test_split
from datautils.ts_dataset import TSDataset
from datautils.data import prepare_dataloaders
import pandas as pd

# Torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

# Transformer
#import tstransformer.Constants as Constants
from tstransformer.Models import Transformer
from tstransformer.Optim import ScheduledOptim

%autoreload 2

In [25]:
# Helper function to determin GPU usage
def get_gpu_memory(name = "GPU Memory", verbose = True):
    import nvidia_smi
    nvidia_smi.nvmlInit()

    
    handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
    info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
    
    memory = {"total":  info.total / (1024 * 1024), "free": info.free / (1024 * 1024), "used": info.used / (1024 * 1024)}
    
    if verbose:
        print("#### ", name , "####")
        print("Total memory:", memory["total"], "MB")
        print("Free memory: ", memory["free"], "MB")
        print("Used memory: ", memory["used"], "MB")
        print("")
    
    nvidia_smi.nvmlShutdown()
    
    return memory

_=get_gpu_memory("Before Empty Cache")
torch.cuda.empty_cache()
_=get_gpu_memory("After Empty Cache")

####  Before Empty Cache ####
Total memory: 12288.0 MB
Free memory:  10347.1875 MB
Used memory:  1940.8125 MB

####  After Empty Cache ####
Total memory: 12288.0 MB
Free memory:  10347.1875 MB
Used memory:  1940.8125 MB



In [26]:
opt = get_options()
print_options(opt)

           train_path :  /home/yannic/master-thesis/data_air/prsa_data.parquet
             val_path :  /home/yannic/master-thesis/data_air/prsa_data.parquet
                epoch :  10
           batch_size :  64
              d_model :  512
       d_inner_hidden :  2048
                d_key :  512
              d_value :  512
           d_sequence :  512
               n_head :  8
             n_layers :  6
       n_warmup_steps :  4000
               lr_mul :  2.0
            loss_func :  rmse
                 seed :  False
              dropout :  0.1
    embs_share_weight :  False
    proj_share_weight :  False
     scale_emb_or_prj :  prj
           output_dir :  ./output
               use_tb :  False
            use_wandb :  False
            save_mode :  best
                 cuda :  True
      label_smoothing :  True


In [32]:
# https://pytorch.org/docs/stable/notes/randomness.html
# For reproducibility
if opt["seed"] is not None:
    torch.manual_seed(opt["seed"])
    torch.backends.cudnn.benchmark = False
    # torch.set_deterministic(True)
    np.random.seed(opt["seed"])
    random.seed(opt["seed"])

In [33]:
if not opt["output_dir"]:
    print('No experiment result will be saved.')
    raise

In [34]:
if not os.path.exists(opt["output_dir"]):
    os.makedirs(opt["output_dir"])

In [35]:
# define the cuda devide
device = torch.device('cuda' if opt["cuda"] else 'cpu')
print(device)

cuda


In [36]:
# Load the dataset and preprocess it 
aq = pd.read_parquet("/home/yannic/master-thesis/data_air/prsa_data.parquet")

# With Stations
#aq_prep = pd.concat([aq,pd.get_dummies(aq['station'], prefix='station',dummy_na=False)],axis=1).drop(['station'],axis=1).drop(columns=["wind_direction"])

# Without Stations
aq_prep = aq.drop(['station'],axis=1).drop(columns=["wind_direction"]).iloc[:,4:]

aq_prep = aq_prep.fillna(0)#[:300]
aq_prep.tail()

,pm25,pm10,so2,no2,co,o3,temperatur,pressure,dew_point,rain,wind_speed
420763,11.0,27.0,4.0,20.0,300.0,81.0,12.6,1011.9,-14.3,0.0,2.0
420764,15.0,43.0,6.0,55.0,500.0,45.0,9.4,1012.3,-11.9,0.0,1.0
420765,13.0,35.0,7.0,48.0,500.0,48.0,8.7,1012.8,-13.7,0.0,1.1
420766,12.0,31.0,5.0,47.0,500.0,50.0,7.8,1012.9,-12.6,0.0,1.0
420767,7.0,25.0,6.0,86.0,700.0,11.0,7.0,1012.6,-11.2,0.0,1.1


# Data Preprocessing

# DataLoader and Transformer

# New Code 

In [37]:
def calc_loss(pred, target, lossfn="rmse"):
    
    # flatten targets in order to match predicitions
    target = target.flatten(start_dim=1)
    
    #TODO: Check if NAN's in target, and mask them out in th eloss calculation
    
    # calc loss
    
    if lossfn=="rmse":
        loss = rmse_loss(pred, target)
        
    elif lossfn=="mse":
        loss = mse_loss(pred, target)
    
    
    
    return loss

def mse_loss(pred, target):
    """" Mean Squared Error loss function """
    loss = F.mse_loss(pred, target)
    
    return loss


def rmse_loss(pred, target):
    """ Root Mean Squared Error loss function """
    loss = torch.sqrt(F.mse_loss(pred, target))
    
    return loss

In [38]:
def train_epoch(model, train_dataloader, optimizer, opt, device):
    
    # Set the model into training mode
    model.train()
    
    # Performance 
    losses = []
    
    desc = '  - (Training)   '
    for data, target in tqdm(train_dataloader, mininterval=2, desc=desc, leave=False, ascii=" ▖▘▝▗▚▞█"): #ascii="░▒█"):
        
        # prepare data
        data = data.to(device)
        target = target.to(device)
        
        # Take the last step of the encoder input sequence to use as a startingpoint for the decoder 
        decoder_input = data[:,-1,:].unsqueeze(dim=1)
        
        # forward pass
        optimizer.zero_grad()
        pred = model(data, decoder_input)
        
        # backward pass
        loss = calc_loss(pred, target, opt["loss_func"])
        loss.backward()
        optimizer.step_and_update_lr()
        
        # Sum up loss
        losses.append(loss)
         
    return losses
    

In [39]:
def train(model, train_dataloader, validation_data, optimizer, device, opt):
    
    # TODO: Implement wandb connection
    
    # TODO: Implement logging
    epoch_losses = []
    
    def print_performance(header, loss_name, loss, start_time, lr):
        print("  - ({header:12}) | {loss_name}: {loss:3.3f} | lr: {lr:8.5f} | elapse: {elapse:3.3f} min".format(header=header,loss_name=loss_name, loss=loss, elapse=(time.time()-start_time)/60, lr=lr))
    
    #TODO: half precission
    
    # TODO: Training Epoch Loop
    for epoch_i in range(opt["epoch"]):
        print("[Epoch: {:>3}]".format(epoch_i))
        
        # trains one epoch
        start = time.time()
        losses = train_epoch(model,train_dataloader,optimizer,opt,device)
        
        #calculate epoch loss and add to logging
        total_epoch_loss = sum([loss.item() for loss in losses])
        epoch_losses.append(total_epoch_loss)
        
        lr = optimizer._optimizer.param_groups[0]['lr']
        
        print_performance('Training',opt["loss_func"], total_epoch_loss, start, lr)
        
        
        #TODO: write function: eval_epoch() 
        #eval_epoch()
    
    

In [40]:
# Load Dataset
opt = get_options()

train_split, test_split = train_test_split(aq_prep, test_size=0.15, shuffle=False)
train_split, eval_split = train_test_split(train_split, test_size=0.15, shuffle=False)

train_dataset, train_dataloader = prepare_dataloaders(train_split.values, opt["batch_size"],window_size=10)
test_dataset, test_dataloader = prepare_dataloaders(test_split.values,  opt["batch_size"],window_size=10)


opt["src_sequence_size"] = 11
opt["trg_sequence_size"] = 11
opt["src_pad_idx"] = 0
opt["trg_pad_idx"] = 0

# Define Transformer
transformer = Transformer(
        n_src_sequence=opt["src_sequence_size"],
        n_trg_sequence=opt["trg_sequence_size"],
        src_pad_idx=opt["src_pad_idx"],
        trg_pad_idx=opt["trg_pad_idx"],
        trg_emb_prj_weight_sharing=opt["proj_share_weight"],
        emb_src_trg_weight_sharing=opt["embs_share_weight"],
        d_k=opt["d_key"],
        d_v=opt["d_value"],
        d_model=opt["d_model"],
        d_sequence_vec=opt["d_sequence"],
        d_inner=opt["d_inner_hidden"],
        n_layers=opt["n_layers"],
        n_head=opt["n_head"],
        dropout=opt["dropout"],
        n_position=opt["d_sequence"],
        scale_emb_or_prj=opt["scale_emb_or_prj"]).to(device)

# Define Optimizer
optimizer = ScheduledOptim(
                optim.Adam(transformer.parameters(), betas=(0.9, 0.98), eps=1e-09),
                opt["lr_mul"], opt["d_model"], opt["n_warmup_steps"])

# Start the training process
#train(transformer, train_dataloader, train_dataloader, optimizer, device, opt)

In [41]:
train(transformer,train_dataloader,train_dataloader,optimizer,device,opt)

[Epoch:   0]


  - (Training    ) | rmse: 2368999.060 | lr:  0.00115 | elapse: 4.202 min
[Epoch:   1]


KeyboardInterrupt: 

# Old Functions

In [ ]:
def cal_performance(pred, gold, trg_pad_idx, smoothing=False):
    ''' Apply label smoothing if needed '''

    loss = cal_loss(pred, gold, trg_pad_idx, smoothing=smoothing)

    pred = pred.max(1)[1]
    gold = gold.contiguous().view(-1)
    non_pad_mask = gold.ne(trg_pad_idx)
    n_correct = pred.eq(gold).masked_select(non_pad_mask).sum().item()
    n_word = non_pad_mask.sum().item()

    return loss, n_correct, n_word

In [ ]:
def cal_loss(pred, gold, trg_pad_idx, smoothing=False):
    ''' Calculate cross entropy loss, apply label smoothing if needed. '''

    gold = gold.contiguous().view(-1)

    if smoothing:
        eps = 0.1
        n_class = pred.size(1)

        one_hot = torch.zeros_like(pred).scatter(1, gold.view(-1, 1), 1)
        one_hot = one_hot * (1 - eps) + (1 - one_hot) * eps / (n_class - 1)
        log_prb = F.log_softmax(pred, dim=1)

        non_pad_mask = gold.ne(trg_pad_idx)
        loss = -(one_hot * log_prb).sum(dim=1)
        loss = loss.masked_select(non_pad_mask).sum()  # average later
    else:
        loss = F.cross_entropy(pred, gold, ignore_index=trg_pad_idx, reduction='sum')
    return loss

In [ ]:
def patch_src(src, pad_idx):
    src = src.transpose(0, 1)
    return src

In [ ]:
def patch_trg(trg, pad_idx):
    trg = trg.transpose(0, 1)
    trg, gold = trg[:, :-1], trg[:, 1:].contiguous().view(-1)
    return trg, gold

In [ ]:
def train_epoch(model, training_data, optimizer, opt, device, smoothing):
    ''' Epoch operation in training phase'''

    model.train()
    total_loss, n_word_total, n_word_correct = 0, 0, 0 

    desc = '  - (Training)   '
    for batch in tqdm(training_data, mininterval=2, desc=desc, leave=False):
        
        
        # prepare data
        src_seq = patch_src(batch.src, opt.src_pad_idx).to(device)
        trg_seq, gold = map(lambda x: x.to(device), patch_trg(batch.trg, opt.trg_pad_idx))

        # forward
        optimizer.zero_grad()
        pred = model(src_seq, trg_seq)

        # backward and update parameters
        loss, n_correct, n_word = cal_performance(
            pred, gold, opt.trg_pad_idx, smoothing=smoothing) 
        loss.backward()
        optimizer.step_and_update_lr()

        # note keeping
        n_word_total += n_word
        n_word_correct += n_correct
        total_loss += loss.item()

    loss_per_word = total_loss/n_word_total
    accuracy = n_word_correct/n_word_total
    return loss_per_word, accuracy

In [ ]:
def eval_epoch(model, validation_data, device, opt):
    ''' Epoch operation in evaluation phase '''

    model.eval()
    total_loss, n_word_total, n_word_correct = 0, 0, 0

    desc = '  - (Validation) '
    with torch.no_grad():
        for batch in tqdm(validation_data, mininterval=2, desc=desc, leave=False):

            # prepare data
            src_seq = patch_src(batch.src, opt.src_pad_idx).to(device)
            trg_seq, gold = map(lambda x: x.to(device), patch_trg(batch.trg, opt.trg_pad_idx))

            # forward
            pred = model(src_seq, trg_seq)
            loss, n_correct, n_word = cal_performance(
                pred, gold, opt.trg_pad_idx, smoothing=False)

            # note keeping
            n_word_total += n_word
            n_word_correct += n_correct
            total_loss += loss.item()

    loss_per_word = total_loss/n_word_total
    accuracy = n_word_correct/n_word_total
    return loss_per_word, accuracy

In [ ]:
def train(model, training_data, validation_data, optimizer, device, opt):
    ''' Start training '''
    
    # Use wandb to plot curves, e.g. perplexity, accuracy, learning rate
    # TODO: Implement this

    log_train_file = os.path.join(opt["output_dir"], 'train.log')
    log_valid_file = os.path.join(opt["output_dir"], 'valid.log')

    print('[Info] Training performance will be written to file: {} and {}'.format(log_train_file, log_valid_file))

    with open(log_train_file, 'w') as log_tf, open(log_valid_file, 'w') as log_vf:
        log_tf.write('epoch,loss,ppl,accuracy\n')
        log_vf.write('epoch,loss,ppl,accuracy\n')

    def print_performances(header, accu, start_time, lr):
        print('  - {header:12} , accuracy: {accu:3.3f} %, lr: {lr:8.5f}, ''elapse: {elapse:3.3f} min'.format(
                  header=f"({header})",accu=100*accu, elapse=(time.time()-start_time)/60, lr=lr))

    #valid_accus = []
    valid_losses = []
    for epoch_i in range(opt["epoch"]):
        print('[ Epoch', epoch_i, ']')

        start = time.time()
        train_loss, train_accu = train_epoch(
            model, training_data, optimizer, opt, device, smoothing=opt["label_smoothing"])
        
        
        # Current learning rate
        lr = optimizer._optimizer.param_groups[0]['lr']
        print_performances('Training', train_accu, start, lr)

        start = time.time()
        valid_loss, valid_accu = eval_epoch(model, validation_data, device, opt)
        valid_ppl = math.exp(min(valid_loss, 100))
        print_performances('Validation', valid_ppl, valid_accu, start, lr)

        valid_losses += [valid_loss]

        checkpoint = {'epoch': epoch_i, 'settings': opt, 'model': model.state_dict()}
        
        if opt["save_mode"] == 'all':
            model_name = 'model_accu_{accu:3.3f}.chkpt'.format(accu=100*valid_accu)
            torch.save(checkpoint, model_name)
        elif opt["save_mode"] == 'best':
            model_name = 'model.chkpt'
            if valid_loss <= min(valid_losses):
                torch.save(checkpoint, os.path.join(opt["output_dir"], model_name))
                print('    - [Info] The checkpoint file has been updated.')

        with open(log_train_file, 'a') as log_tf, open(log_valid_file, 'a') as log_vf:
            log_tf.write('{epoch},{loss: 8.5f},{ppl: 8.5f},{accu:3.3f}\n'.format(
                epoch=epoch_i, loss=train_loss,
                ppl=train_ppl, accu=100*train_accu))
            log_vf.write('{epoch},{loss: 8.5f},{ppl: 8.5f},{accu:3.3f}\n'.format(
                epoch=epoch_i, loss=valid_loss,
                ppl=valid_ppl, accu=100*valid_accu))

        if opt["use_tb"]:
            tb_writer.add_scalars('ppl', {'train': train_ppl, 'val': valid_ppl}, epoch_i)
            tb_writer.add_scalars('accuracy', {'train': train_accu*100, 'val': valid_accu*100}, epoch_i)
            tb_writer.add_scalar('learning_rate', lr, epoch_i)


--------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Testing and Stuff